In [1]:
import numpy as np 
import cv2
from ultralytics import YOLO, NAS
import supervision as sv
import os
import torch.onnx


In [2]:
detector = YOLO("models/yolov8s_people.onnx")
classifier = YOLO("models/yolov8s_pose_classifier.pt")
byte_tracker = sv.ByteTrack()

WARNING  Unable to automatically guess model task, assuming 'task=detect'. Explicitly define task for your model, i.e. 'task=detect', 'segment', 'classify', or 'pose'.


In [15]:
# from super_gradients.training import models

# detector = models.get("yolo_nas_s", pretrained_weights="coco")

In [10]:
res = detector.predict("data/stand.jpg")[0]

In [14]:
res.prediction.bboxes_xyxy

array([[     696.01,       438.8,      878.72,      625.33],
       [     409.84,      182.03,      518.02,      438.12],
       [     878.24,      477.37,      1103.1,      626.04],
       [      782.5,      333.12,      1002.2,      465.29],
       [     924.01,      335.14,      1086.6,      519.74],
       [     727.09,      296.32,      821.16,      479.57],
       [     884.75,      333.79,      1090.4,      572.29],
       [     856.28,      318.67,      885.03,      378.11]], dtype=float32)

In [25]:
detector.model.eval()

DetectionModel(
  (model): Sequential(
    (0): Conv(
      (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
      (act): SiLU(inplace=True)
    )
    (1): Conv(
      (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(64, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
      (act): SiLU(inplace=True)
    )
    (2): C2f(
      (cv1): Conv(
        (conv): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn): BatchNorm2d(64, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (cv2): Conv(
        (conv): Conv2d(96, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn): BatchNorm2d(64, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
    

In [26]:
detector.export(format="onnx", imgsz=[640,640],  opset=12, nms=True)

Ultralytics YOLOv8.0.209  Python-3.8.9 torch-2.1.0+cpu CPU (11th Gen Intel Core(TM) i5-1145G7 2.60GHz)
Model summary (fused): 168 layers, 11125971 parameters, 0 gradients, 28.4 GFLOPs

PyTorch: starting from 'models\people_detector_yolov8s.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 5, 8400) (21.5 MB)

ONNX: starting export with onnx 1.15.0 opset 12...
ONNX: export success  1.4s, saved as 'models\people_detector_yolov8s.onnx' (42.6 MB)

Export complete (3.6s)
Results saved to C:\Projects\PeopleFallDetection\models
Predict:         yolo predict task=detect model=models\people_detector_yolov8s.onnx imgsz=640  
Validate:        yolo val task=detect model=models\people_detector_yolov8s.onnx imgsz=640 data=gdrive/MyDrive/yolov8_configs/yolo_data.yaml  
Visualize:       https://netron.app


'models\\people_detector_yolov8s.onnx'

In [3]:
classes = ["fall", "no_fall"]

In [4]:
MAX_FALL=30
MAX_NOT_MOVING = 20

In [5]:
res = classifier("data/stand.jpg")


image 1/1 c:\Projects\PeopleFallDetection\data\stand.jpg: 224x224 no fall 0.67, sitting 0.30, fall 0.03, 44.7ms
Speed: 7.3ms preprocess, 44.7ms inference, 0.0ms postprocess per image at shape (1, 3, 224, 224)


In [8]:
res[0].probs.data

tensor([0.0294, 0.6745, 0.2962])

In [6]:
def compute_offsets(bbox1, bbox2):
    '''
    compute differences between previous and current bboxes
    '''
    dx = (bbox2[0]-bbox1[0])**2 + (bbox2[2]-bbox1[2])**2
    dy = (bbox2[1]-bbox1[1])**2 + (bbox2[3]-bbox1[3])**2
    
    return dx+dy

In [9]:
if compute_offsets([2,3, 5,6], [2,3, 5,6])== 0:
    print("true")

true


In [15]:
#model from s3
detector = YOLO("models/yolov8s_people.onnx", task='detect')

### Using yolov8 model and classificator for predictions

In [10]:
cap = cv2.VideoCapture("data/people_falling_3.mp4")

frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

scale_percent = 50 # percent of original size
width = int(frame_width * scale_percent / 100)
height = int(frame_height * scale_percent / 100)


cap_writer = cv2.VideoWriter('data/result_detection.avi', cv2.VideoWriter_fourcc(*'MJPG'),
                        30, (width, height))

success, frame = cap.read()
frame_num=0
total_fps=0

frame_num=0

OBJECTS={}

while success:
    
    
    width = int(frame.shape[1] * scale_percent / 100)
    height = int(frame.shape[0] * scale_percent / 100)
    
    dim = (width, height)
  
    # resize image
    frame = cv2.resize(frame, dim, interpolation = cv2.INTER_AREA)
    
        
    alarm = False
    
    for object in OBJECTS.values():
            object["detected"] = False
    
       
    if frame_num%3==0:
        #renew predictions every 5 frame
        detections = detector(frame)[0]
        detections = sv.Detections.from_ultralytics(detections)  
                
    detections = byte_tracker.update_with_detections(detections)
        
       
    bboxes = detections.xyxy
    confidences = detections.confidence
    class_ids = detections.class_id
    tracker_ids = detections.tracker_id
    
            
    for pred_box, label, tracker_id in zip(bboxes, class_ids, tracker_ids):
        
        if label == 0:
            bbox = list(map(int, pred_box))
            width = bbox[2]-bbox[0]
            height = bbox[3]-bbox[1]
            ROI = frame[bbox[1]:bbox[1]+height, 
                    bbox[0]:bbox[0]+width]
        
            results = classifier(ROI)
            second_class = classes[np.argmax(results[0].probs.data)]

            if tracker_id not in OBJECTS:
                OBJECTS[tracker_id] = {
                    "class_id": label,
                    "last_bbox": np.array(bbox),
                    "bbox": np.array(bbox),
                    "second_class": second_class,
                    "fall": 0,
                    "bbox_offsets": 0,
                    "not_moving": 0,
                    "detected": True
                    
                }
            else:
                object = OBJECTS[tracker_id] 
                object['class_id']=label
                object['last_bbox'] = OBJECTS[tracker_id]['bbox']
                object['bbox']=np.array(bbox)
                object['second_class']=second_class
                object['bbox_offsets'] = compute_offsets(object['last_bbox'], object['bbox'])
                if object['second_class']=="fall": #if fall
                    object['fall']+=1
                    if object['bbox_offsets']<=2:
                        object['not_moving']+=1
                    else:
                        object['not_moving']=object['not_moving']-1 if object['not_moving']>0 else 0   
                else:
                    object['fall']=0    
                    object['not_moving']=0
                object['detected']=True    
        
            
            cv2.rectangle(frame, (bbox[0], bbox[1]), (bbox[2], bbox[3]), color=(0,0,255), thickness=2)
            cv2.putText(frame, "id: "+str(tracker_id), (bbox[0], bbox[1]-10), 
                        cv2.FONT_HERSHEY_SIMPLEX , fontScale = 0.5,
                        color=(0, 0, 0), thickness=1)
            cv2.putText(frame, second_class, (bbox[0]+50, bbox[1]-10),
                        cv2.FONT_HERSHEY_SIMPLEX , fontScale = 0.5,
                        color=(0, 0, 0), thickness=1)
            
        
        OBJECTS_copy = OBJECTS.copy()
        for object_id in OBJECTS_copy.keys():
            if OBJECTS[object_id]["detected"] == False:
                del OBJECTS[object_id]
        
        #check falling objects
        for ind, object in OBJECTS.items():
            if object['fall']>=MAX_FALL:
                 cv2.putText(frame, "ALARM!", (20, 50), 
                             cv2.FONT_HERSHEY_SIMPLEX , fontScale = 1, 
                             color=(0, 0, 255), thickness=5) 
                 if object['not_moving']>=MAX_NOT_MOVING:
                     cv2.putText(frame, "Not moving!", (150, 50), 
                             cv2.FONT_HERSHEY_SIMPLEX , fontScale = 1, 
                             color=(0, 0, 255), thickness=3)
        
             
                
                    
    #resize frame for videowriter
    #frame = cv2.resize(frame, dsize=(640, 640), interpolation=cv2.INTER_CUBIC) 
    cap_writer.write(frame)      
    cv2.imshow("frame", frame)
    
    key = cv2.waitKey(1)
    if key == ord('q'):
        cap.release()
        cv2.destroyAllWindows()
        break
    
    success, frame = cap.read()
    frame_num+=1
    




0: 640x640 3 persons, 193.0ms
Speed: 5.1ms preprocess, 193.0ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 3 persons, 134.3ms
Speed: 5.7ms preprocess, 134.3ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 3 persons, 116.2ms
Speed: 5.8ms preprocess, 116.2ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 3 persons, 139.6ms
Speed: 3.3ms preprocess, 139.6ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 3 persons, 144.9ms
Speed: 0.0ms preprocess, 144.9ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 3 persons, 154.8ms
Speed: 4.0ms preprocess, 154.8ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 3 persons, 146.9ms
Speed: 3.0ms preprocess, 146.9ms inference, 15.6ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 3 persons, 110.9ms
Speed: 3.4ms preprocess, 110.9ms inference, 0.0ms postprocess per

In [ ]:
cap = cv2.VideoCapture("data/people_falling_3.mp4")

frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

scale_percent = 50 # percent of original size
width = int(frame_width * scale_percent / 100)
height = int(frame_height * scale_percent / 100)


cap_writer = cv2.VideoWriter('data/result_detection.avi', cv2.VideoWriter_fourcc(*'MJPG'),
                        30, (width, height))

success, frame = cap.read()
frame_num=0
total_fps=0

frame_num=0

OBJECTS={}

while success:
    
    
    width = int(frame.shape[1] * scale_percent / 100)
    height = int(frame.shape[0] * scale_percent / 100)
    
    dim = (width, height)
  
    # resize image
    frame = cv2.resize(frame, dim, interpolation = cv2.INTER_AREA)
    
        
    alarm = False
    
    for object in OBJECTS.values():
            object["detected"] = False
    
       
    if frame_num%3==0:
        #renew predictions every 5 frame
        detections = detector(frame)[0]
        detections = sv.Detections.from_ultralytics(detections)  
                
    detections = byte_tracker.update_with_detections(detections)
        
       
    bboxes = detections.xyxy
    confidences = detections.confidence
    class_ids = detections.class_id
    tracker_ids = detections.tracker_id
    
            
    for pred_box, label, tracker_id in zip(bboxes, class_ids, tracker_ids):
        
        if label == 0:
            bbox = list(map(int, pred_box))
            width = bbox[2]-bbox[0]
            height = bbox[3]-bbox[1]
            ROI = frame[bbox[1]:bbox[1]+height, 
                    bbox[0]:bbox[0]+width]
        
            results = classifier(ROI)
            second_class = classes[np.argmax(results[0].probs.data)]
            
            
            if tracker_id not in OBJECTS:
                OBJECTS[tracker_id] = {
                    "class_id": label,
                    "last_bbox": np.array(bbox),
                    "bbox": np.array(bbox),
                    "second_class": second_class,
                    "fall": 0,
                    "bbox_offsets": 0,
                    "not_moving": 0,
                    "detected": True
                    
                }
            else:
                object = OBJECTS[tracker_id] 
                object['class_id']=label
                object['last_bbox'] = OBJECTS[tracker_id]['bbox']
                object['bbox']=np.array(bbox)
                object['second_class']=second_class
                object['bbox_offsets'] = compute_offsets(object['last_bbox'], object['bbox'])
                if object['second_class']=="fall": #if fall
                    object['fall']+=1
                    if object['bbox_offsets']<=2:
                        object['not_moving']+=1
                    else:
                        object['not_moving']=object['not_moving']-1 if object['not_moving']>0 else 0   
                else:
                    object['fall']=0    
                    object['not_moving']=0
                object['detected']=True    
        
            
            cv2.rectangle(frame, (bbox[0], bbox[1]), (bbox[2], bbox[3]), color=(0,0,255), thickness=2)
            cv2.putText(frame, "id: "+str(tracker_id), (bbox[0], bbox[1]-10), 
                        cv2.FONT_HERSHEY_SIMPLEX , fontScale = 0.5,
                        color=(0, 0, 0), thickness=1)
            cv2.putText(frame, second_class, (bbox[0]+50, bbox[1]-10),
                        cv2.FONT_HERSHEY_SIMPLEX , fontScale = 0.5,
                        color=(0, 0, 0), thickness=1)
            
        
        OBJECTS_copy = OBJECTS.copy()
        for object_id in OBJECTS_copy.keys():
            if OBJECTS[object_id]["detected"] == False:
                del OBJECTS[object_id]
        
        #check falling objects
        for ind, object in OBJECTS.items():
            if object['fall']>=MAX_FALL:
                 cv2.putText(frame, "ALARM!", (20, 50), 
                             cv2.FONT_HERSHEY_SIMPLEX , fontScale = 1, 
                             color=(0, 0, 255), thickness=5) 
                 if object['not_moving']>=MAX_NOT_MOVING:
                     cv2.putText(frame, "Not moving!", (150, 50), 
                             cv2.FONT_HERSHEY_SIMPLEX , fontScale = 1, 
                             color=(0, 0, 255), thickness=3)
        
             
                
                    
    #resize frame for videowriter
    #frame = cv2.resize(frame, dsize=(640, 640), interpolation=cv2.INTER_CUBIC) 
    cap_writer.write(frame)      
    cv2.imshow("frame", frame)
    
    key = cv2.waitKey(1)
    if key == ord('q'):
        cap.release()
        cv2.destroyAllWindows()
        break
    
    success, frame = cap.read()
    frame_num+=1
    


In [16]:
OBJECTS

{18: {'class_id': 0,
  'last_bbox': array([229,  29, 323, 251]),
  'bbox': array([229,  29, 322, 251]),
  'second_class': 'standing',
  'fall': 0,
  'bbox_offsets': 1,
  'not_moving': 0,
  'detected': True}}

In [26]:
for object in OBJECTS.items():
    print(object)

(231, {'class_id': 0, 'bbox': array([203, 101, 384, 435]), 'second_class': 'standing', 'fall': 0})


In [2]:
d={"1":1, "2":2}

In [3]:
d.values()

dict_values([1, 2])

### Extract frames from video

In [2]:
def make_frames_from_video(file_name, output_dir):

    os.mkdir(output_dir)
    

    video_path = os.path.join(".", file_name)


    cap = cv2.VideoCapture(video_path)
    ret, frame = cap.read()
    frame_ind=0
    while ret:
        frame_ind+=1
        cv2.imwrite(f"{output_dir}/{output_dir}_{frame_ind}.jpg", frame)
        ret, frame = cap.read()


    cap.release()
    cv2.destroyAllWindows()

In [10]:
make_frames_from_video("data/me.mp4", "frames")

In [29]:
make_frames_from_video("data/me2.mp4", "frames_me2")

In [30]:
make_frames_from_video("data/not_moving1.mp4", "frames_not_moving1")

In [31]:
make_frames_from_video("data/not_moving2.mp4", "frames_not_moving2")

In [32]:
make_frames_from_video("data/not_moving3.mp4", "frames_not_moving3")

In [33]:
make_frames_from_video("data/not_moving4.mp4", "frames_not_moving4")

In [34]:
make_frames_from_video("data/not_moving5.mp4", "frames_not_moving5")

In [35]:
make_frames_from_video("data/people_falling_1.mp4", "frames_people_falling_1")

In [29]:
make_frames_from_video("data/people_falling_2.mp4", "frames_people_falling_2")

In [30]:
make_frames_from_video("data/people_falling_3.mp4", "frames_people_falling_3")

In [31]:
make_frames_from_video("data/people_sitting_1.mp4", "frames_people_sitting_1")

In [32]:
make_frames_from_video("data/me3.mp4", "frames_me3")

In [33]:
make_frames_from_video("data/me4.mp4", "frames_me4")

In [4]:
make_frames_from_video("data/istock_faint.mp4", "frames_istock_faint")

### Train YOLO people detector on coco subset + frames from my video

In [2]:
model = YOLO("yolov8s.pt")

In [3]:
model.train(data="yolo_data.yaml", cfg="yolov8_hyperparameters.yaml", 
            epochs=100, batch=16, imgsz=640, pretrained=True, project="yolov8n-people-detection")

New https://pypi.org/project/ultralytics/8.0.215 available  Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.0.209  Python-3.8.9 torch-2.1.0+cpu CPU (11th Gen Intel Core(TM) i5-1145G7 2.60GHz)
engine\trainer: task=detect, mode=train, model=None, data=yolo_data.yaml, epochs=100, patience=50, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=yolov8n-people-detection, name=train2, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=20, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, stream_buffer=False, line_width=None, visualize=False

KeyboardInterrupt: 

### Training YOLO cls

In [8]:
DATASET_FOLDER = "datasets"

In [6]:
model = YOLO("yolov8s-cls.pt")

100%|██████████| 12.2M/12.2M [00:00<00:00, 14.7MB/s]


In [9]:
model.train(data=DATASET_FOLDER, epochs=50, imgsz=224)

New https://pypi.org/project/ultralytics/8.0.211 available  Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.0.209  Python-3.8.9 torch-2.1.0+cpu CPU (11th Gen Intel Core(TM) i5-1145G7 2.60GHz)
engine\trainer: task=classify, mode=train, model=yolov8s-cls.pt, data=datasets, epochs=50, patience=50, batch=16, imgsz=224, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train2, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, stream_buffer=False, line_width=None, visualize=False, augment=False

KeyboardInterrupt: 

In [3]:
!yolo classify train imgsz=224 epochs=50 data={DATASET_FOLDER} model=yolov8s-cls.pt --project yolov8s-people_fall_3_classes device=0

WARNING  '--project' does not require leading dashes '--', updating to 'project'.
Traceback (most recent call last):
  File "C:\Users\Alena_Prakonina\AppData\Local\Programs\Python\Python38\lib\runpy.py", line 194, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "C:\Users\Alena_Prakonina\AppData\Local\Programs\Python\Python38\lib\runpy.py", line 87, in _run_code
    exec(code, run_globals)
  File "c:\Projects\PeopleFallDetection\.venv\Scripts\yolo.exe\__main__.py", line 7, in <module>
  File "c:\projects\peoplefalldetection\.venv\lib\site-packages\ultralytics\cfg\__init__.py", line 378, in entrypoint
    raise SyntaxError(f"'{colorstr('red', 'bold', a)}' is a valid YOLO argument but is missing an '=' sign "
SyntaxError: 'project' is a valid YOLO argument but is missing an '=' sign to set its value, i.e. try 'project=None'

    Arguments received: ['yolo', 'classify', 'train', 'imgsz=224', 'epochs=50', 'data=datasets/train/', 'model=yolov8s-cls.pt', '--projec

In [2]:
def check_not_moving(frame, not_moving):
    if not_moving>=10:
        cv2.putText(frame, "Not moving",
                    (150, 50),
                    cv2.FONT_HERSHEY_SIMPLEX , fontScale = 1, 
                             color=(0, 0, 255), thickness=3) 

In [4]:
cap = cv2.VideoCapture("data/me.mp4")

frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

scale_percent = 50 # percent of original size
width = int(frame_width * scale_percent / 100)
height = int(frame_height * scale_percent / 100)
    
backSub = cv2.createBackgroundSubtractorMOG2()

not_moving=0

while(cap.isOpened()):
    
    success, frame_prev = cap.read()
      
    
    fgMask_prev = backSub.apply(frame_prev)
    
    frame_prev = cv2.resize(frame_prev, dsize=(width, height))
    fgMask_prev = cv2.resize(fgMask_prev, dsize=(width, height))
    
    check_not_moving(frame_prev, not_moving) 
    
    cv2.imshow('Frame', frame_prev)
    cv2.imshow('FG Mask', fgMask_prev)
    
    
    success, frame_next = cap.read()
    
    fgMask_next = backSub.apply(frame_next)
    
    frame_next = cv2.resize(frame_next, dsize=(width, height))
    fgMask_next = cv2.resize(fgMask_next, dsize=(width, height))
    
    diff = cv2.absdiff(fgMask_prev, fgMask_next)   
    thresh_diff = cv2.threshold(diff, 15, 255, cv2.THRESH_BINARY)[1]
    
    
    cv2.imshow("difference", diff)
    
    # Calculate the difference between the 2 images
    total_pixels = frame_prev.shape[0] * frame_prev.shape[1] * 1.0
    diff_on_pixels = cv2.countNonZero(thresh_diff) * 1.0
    difference_measure = diff_on_pixels / total_pixels
    
    print(difference_measure)
    
    if difference_measure<=0.03:
        not_moving+=1   
    else:
        not_moving = 0    
          
    check_not_moving(frame_next, not_moving) 
    
       
    cv2.imshow('Frame', frame_next)
    cv2.imshow('FG Mask', fgMask_next)
    
    key = cv2.waitKey(1)
    if key == ord('q'):
        cap.release()
        cv2.destroyAllWindows()
        break
    
    
    success, frame = cap.read()
    

0.9987037037037036
0.03327739197530864
0.0238695987654321
0.01863425925925926
0.03042631172839506
0.023227237654320988
0.01912229938271605
0.020482253086419752
0.028512731481481483
0.02947530864197531
0.030495756172839505
0.03898533950617284
0.026786265432098767
0.03362654320987654
0.027662037037037037
0.03857638888888889
0.0510570987654321
0.05089699074074074
0.06965663580246914
0.08076774691358024
0.0691724537037037
0.07252893518518519
0.06911651234567902
0.07480902777777777
0.06703317901234568
0.06967785493827161
0.06793788580246914
0.0655420524691358
0.06500964506172839
0.06569251543209877
0.0619579475308642
0.0575462962962963
0.06144868827160494
0.06543016975308642
0.0654320987654321
0.060271990740740744
0.06544753086419754
0.06255979938271605
0.06515432098765432
0.06457947530864197
0.057629243827160496
0.05886766975308642
0.06327160493827161
0.06466242283950617
0.0619849537037037
0.06371527777777777
0.06276427469135802
0.06542631172839507
0.06360725308641975
0.0676466049382716
0.

error: OpenCV(4.8.1) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\resize.cpp:4062: error: (-215:Assertion failed) !ssize.empty() in function 'cv::resize'


: 

In [25]:
def check_extensions(dir):
    extensions=[]
    for root, directories, files in os.walk("datasets_fall_no_fall"):
        print(f"Directory: {root}")
        for file in files:
            extensions.append(file.split(".")[-1])
            
    return set(extensions)        
    

In [26]:
extensions = check_extensions("datasets_fall_no_fall")            

Directory: datasets_fall_no_fall
Directory: datasets_fall_no_fall\test
Directory: datasets_fall_no_fall\test\fall
Directory: datasets_fall_no_fall\test\no fall
Directory: datasets_fall_no_fall\train
Directory: datasets_fall_no_fall\train\fall
Directory: datasets_fall_no_fall\train\no fall
Directory: datasets_fall_no_fall\valid
Directory: datasets_fall_no_fall\valid\fall
Directory: datasets_fall_no_fall\valid\no fall


In [27]:
extensions

{'cache', 'jpg', 'png'}

### Pose estimation yolov8 with keypoints

In [36]:
detector = YOLO("models/yolov8s_people.onnx")
classifier = YOLO("models/yolov8s_pose_classifier.pt")
keypoints_detector = YOLO('yolov8s-pose.pt')

WARNING  Unable to automatically guess model task, assuming 'task=detect'. Explicitly define task for your model, i.e. 'task=detect', 'segment', 'classify', or 'pose'.


In [19]:
results = keypoints_detector('frames_people_falling_1/frames_people_falling_1_1 - Copy.jpg')[0]

# Extract keypoint
result_keypoint = results.keypoints.xyn.cpu().numpy()


image 1/1 c:\Projects\PeopleFallDetection\frames_people_falling_1\frames_people_falling_1_1 - Copy.jpg: 384x640 3 persons, 140.2ms
Speed: 2.0ms preprocess, 140.2ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)


In [24]:
result_keypoint

array([[[    0.68457,     0.21763],
        [    0.69033,     0.20645],
        [    0.67731,     0.20902],
        [    0.69849,     0.20838],
        [    0.66526,      0.2161],
        [    0.71295,     0.27837],
        [    0.65318,     0.28311],
        [    0.73248,     0.36703],
        [    0.63897,     0.36874],
        [     0.7289,     0.44993],
        [    0.63793,     0.45498],
        [    0.70439,     0.46213],
        [    0.66563,     0.46216],
        [    0.70177,     0.58595],
        [    0.66087,     0.58695],
        [    0.70201,     0.71279],
        [     0.6647,     0.70015]],

       [[    0.28833,     0.22953],
        [    0.29485,     0.21835],
        [    0.28009,       0.222],
        [    0.30764,      0.2288],
        [    0.27165,     0.23783],
        [    0.32987,     0.30543],
        [    0.26696,     0.31923],
        [    0.35107,     0.38925],
        [    0.26444,      0.4065],
        [    0.31475,     0.44137],
        [     0.2892,     

In [23]:
results.keypoints.conf

tensor([[0.9859, 0.9538, 0.9516, 0.7374, 0.7583, 0.9951, 0.9960, 0.9804, 0.9844, 0.9734, 0.9774, 0.9980, 0.9981, 0.9940, 0.9937, 0.9759, 0.9749],
        [0.9878, 0.9742, 0.9467, 0.8620, 0.6968, 0.9961, 0.9937, 0.9808, 0.9634, 0.9682, 0.9508, 0.9977, 0.9972, 0.9918, 0.9891, 0.9594, 0.9514],
        [0.9246, 0.7434, 0.9502, 0.2675, 0.9078, 0.9011, 0.9911, 0.2329, 0.9404, 0.1911, 0.8310, 0.9281, 0.9829, 0.8572, 0.9690, 0.7830, 0.9262]])

In [13]:
len(result_keypoint)

3

In [22]:
def plot_skeleton_kpts(im, kpts, confidence):
    
    height, width = im.shape[:2]
    
      
    
    #Plot the skeleton and keypointsfor coco datatset
    palette = np.array([[255, 128, 0], [255, 153, 51], [255, 178, 102],
                        [230, 230, 0], [255, 153, 255], [153, 204, 255],
                        [255, 102, 255], [255, 51, 255], [102, 178, 255],
                        [51, 153, 255], [255, 153, 153], [255, 102, 102],
                        [255, 51, 51], [153, 255, 153], [102, 255, 102],
                        [51, 255, 51], [0, 255, 0], [0, 0, 255], [255, 0, 0],
                        [255, 255, 255]])

    skeleton = [[16, 14], [14, 12], [17, 15], [15, 13], [12, 13], [6, 12],
                [7, 13], [6, 7], [6, 8], [7, 9], [8, 10], [9, 11], [2, 3],
                [1, 2], [1, 3], [2, 4], [3, 5], [4, 6], [5, 7]]

    pose_limb_color = palette[[9, 9, 9, 9, 7, 7, 7, 0, 0, 0, 0, 0, 16, 16, 16, 16, 16, 16, 16]]
    pose_kpt_color = palette[[16, 16, 16, 16, 16, 0, 0, 0, 0, 0, 0, 9, 9, 9, 9, 9, 9]]
    radius = 5
        
    for kpts_list, conf_list in zip(kpts, confidence):
        
        num_kpts = len(kpts_list)

        for kid in range(num_kpts):
            r, g, b = pose_kpt_color[kid]
            x_coord, y_coord = kpts_list[kid][0]*width, kpts_list[kid][1]*height
            conf = conf_list[kid]
            if conf < 0.5:
                continue
            cv2.circle(im, (int(x_coord), int(y_coord)), radius, (int(r), int(g), int(b)), -1)

        for sk_id, sk in enumerate(skeleton):
            r, g, b = pose_limb_color[sk_id]
            pos1 = (int(kpts_list[sk[0]-1][0]*width), int(kpts_list[sk[0]-1][1]*height))
            pos2 = (int(kpts_list[sk[1]-1][0]*width), int(kpts_list[sk[1]-1][1]*height))
            conf1 = conf_list[sk[0]-1]
            conf2 = conf_list[sk[1]-1]
            if conf1<0.5 or conf2<0.5:
                    continue
            cv2.line(im, pos1, pos2, (int(r), int(g), int(b)), thickness=2)
            
    return im        

In [43]:
def plot_skeleton_kpts_on_ROI(im, kpts, confidence, ROI, bbox):
    
    height, width = ROI.shape[:2]
    
    x_start, y_start = bbox[:2]
    
      
    
    #Plot the skeleton and keypointsfor coco datatset
    palette = np.array([[255, 128, 0], [255, 153, 51], [255, 178, 102],
                        [230, 230, 0], [255, 153, 255], [153, 204, 255],
                        [255, 102, 255], [255, 51, 255], [102, 178, 255],
                        [51, 153, 255], [255, 153, 153], [255, 102, 102],
                        [255, 51, 51], [153, 255, 153], [102, 255, 102],
                        [51, 255, 51], [0, 255, 0], [0, 0, 255], [255, 0, 0],
                        [255, 255, 255]])

    skeleton = [[16, 14], [14, 12], [17, 15], [15, 13], [12, 13], [6, 12],
                [7, 13], [6, 7], [6, 8], [7, 9], [8, 10], [9, 11], [2, 3],
                [1, 2], [1, 3], [2, 4], [3, 5], [4, 6], [5, 7]]

    pose_limb_color = palette[[9, 9, 9, 9, 7, 7, 7, 0, 0, 0, 0, 0, 16, 16, 16, 16, 16, 16, 16]]
    pose_kpt_color = palette[[16, 16, 16, 16, 16, 0, 0, 0, 0, 0, 0, 9, 9, 9, 9, 9, 9]]
    radius = 5
        
    for kpts_list, conf_list in zip(kpts, confidence):
        
        num_kpts = len(kpts_list)

        for kid in range(num_kpts):
            r, g, b = pose_kpt_color[kid]
            x_coord, y_coord = x_start+kpts_list[kid][0]*width, y_start+kpts_list[kid][1]*height
            conf = conf_list[kid]
            if conf < 0.5:
                continue
            cv2.circle(im, (int(x_coord), int(y_coord)), radius, (int(r), int(g), int(b)), -1)

        for sk_id, sk in enumerate(skeleton):
            r, g, b = pose_limb_color[sk_id]
            pos1 = (int(x_start+kpts_list[sk[0]-1][0]*width), int(y_start+kpts_list[sk[0]-1][1]*height))
            pos2 = (int(x_start+kpts_list[sk[1]-1][0]*width), int(y_start+kpts_list[sk[1]-1][1]*height))
            conf1 = conf_list[sk[0]-1]
            conf2 = conf_list[sk[1]-1]
            if conf1<0.5 or conf2<0.5:
                    continue
            cv2.line(im, pos1, pos2, (int(r), int(g), int(b)), thickness=2)
            
    return im        

In [29]:
def compute_kpts_difference(kpts1, kpts2):
    if kpts1.shape[1]==kpts2.shape[1]:
        if kpts1.shape[0]>1:
            kpts1=kpts1[0]
        if kpts2.shape[0]>1:
            kpts2=kpts2[0]    
        return np.linalg.norm(kpts1 - kpts2)
    else:
        return 100

In [45]:
cap = cv2.VideoCapture("data/me_cut.mp4")

frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

scale_percent = 50 # percent of original size
width = int(frame_width * scale_percent / 100)
height = int(frame_height * scale_percent / 100)


cap_writer = cv2.VideoWriter('data/result_detection.avi', cv2.VideoWriter_fourcc(*'MJPG'),
                        30, (width, height))

success, frame = cap.read()
frame_num=0
total_fps=0

OBJECTS={}

while success:
    
    
    width = int(frame.shape[1] * scale_percent / 100)
    height = int(frame.shape[0] * scale_percent / 100)
    
    dim = (width, height)
  
    # resize image
    frame = cv2.resize(frame, dim, interpolation = cv2.INTER_AREA)
    
        
    alarm = False
    
    for object in OBJECTS.values():
            object["detected"] = False
    
       
    if frame_num%1==0:
        #renew predictions every 5 frame
        detections = detector(frame)[0]
        detections = sv.Detections.from_ultralytics(detections)  
                
    detections = byte_tracker.update_with_detections(detections)
    
            
       
    bboxes = detections.xyxy
    confidences = detections.confidence
    class_ids = detections.class_id
    tracker_ids = detections.tracker_id
    
            
    for pred_box, label, tracker_id in zip(bboxes, class_ids, tracker_ids):
        
        if label == 0:
            bbox = list(map(int, pred_box))
            ROI = frame[bbox[1]:bbox[3], 
                    bbox[0]:bbox[2]]
        
            results = classifier(ROI)
            second_class = classes[np.argmax(results[0].probs.data)]
            
            
            # results_frame = keypoints_detector(frame)[0]
            # kpts_frame = results_frame.keypoints.xyn.cpu().numpy()
            # confidence = results_frame.keypoints.conf
            # print(kpts_frame.shape)
            # try:
            #     frame = plot_skeleton_kpts(frame, kpts_frame, confidence)
            # except:
            #     Exception("TypeError")
            #     continue
            
            #detect keypoints for ROI
            results_ROI = keypoints_detector(ROI)[0]
            kpts = results_ROI.keypoints.xyn.cpu().numpy()
            confidence = results_ROI.keypoints.conf
            
            # print(kpts.shape)
            try:
                frame = plot_skeleton_kpts_on_ROI(frame, kpts, confidence, ROI, bbox)
            except:
                Exception("TypeError")
                continue       
            
            #if second_class == "fall":
                # results = keypoints_detector(ROI)[0]
                # kpts = results.keypoints.xyn.cpu().numpy()
                # confidence = results.keypoints.conf
                
                # frame = plot_skeleton_kpts(frame, kpts, confidence)

            if tracker_id not in OBJECTS:
                OBJECTS[tracker_id] = {
                    "class_id": label,
                    "last_bbox": np.array(bbox),
                    "bbox": np.array(bbox),
                    'last_kpts': np.array(kpts),
                    'kpts': np.array(kpts),
                    "second_class": second_class,
                    "fall": 0,
                    "kpts_offsets": 0,
                    "not_moving": 0,
                    "detected": True
                    
                }
            else:
                object = OBJECTS[tracker_id] 
                object['class_id']=label
                object['last_bbox'] = OBJECTS[tracker_id]['bbox']
                object['bbox']=np.array(bbox)
                object['second_class']=second_class
                object['last_kpts'] = OBJECTS[tracker_id]['kpts']
                object['kpts'] = np.array(kpts)
                object['kpts_offsets'] = compute_kpts_difference(object['last_kpts'], object['kpts'])
                print(object['kpts_offsets'])
                if object['second_class']=="fall": #if fall
                    object['fall']+=1
                    if object['kpts_offsets']<=2:
                        object['not_moving']+=1
                    else:
                        object['not_moving']=object['not_moving']-1 if object['not_moving']>0 else 0   
                else:
                    object['fall']=0    
                    object['not_moving']=0
                object['detected']=True    
        
            
            cv2.rectangle(frame, (bbox[0], bbox[1]), (bbox[2], bbox[3]), color=(0,0,255), thickness=2)
            cv2.putText(frame, "id: "+str(tracker_id), (bbox[0], bbox[1]-10), 
                        cv2.FONT_HERSHEY_SIMPLEX , fontScale = 0.5,
                        color=(0, 0, 0), thickness=1)
            cv2.putText(frame, second_class, (bbox[0]+50, bbox[1]-10),
                        cv2.FONT_HERSHEY_SIMPLEX , fontScale = 0.5,
                        color=(0, 0, 0), thickness=1)
            
        
        OBJECTS_copy = OBJECTS.copy()
        for object_id in OBJECTS_copy.keys():
            if OBJECTS[object_id]["detected"] == False:
                del OBJECTS[object_id]
        
        #check falling objects
        for ind, object in OBJECTS.items():
            if object['fall']>=MAX_FALL:
                 cv2.putText(frame, "ALARM!", (20, 50), 
                             cv2.FONT_HERSHEY_SIMPLEX , fontScale = 1, 
                             color=(0, 0, 255), thickness=5) 
                 if object['not_moving']>=MAX_NOT_MOVING:
                     cv2.putText(frame, "Not moving!", (150, 50), 
                             cv2.FONT_HERSHEY_SIMPLEX , fontScale = 1, 
                             color=(0, 0, 255), thickness=3)
        
             
                
                    
    #resize frame for videowriter
    #frame = cv2.resize(frame, dsize=(640, 640), interpolation=cv2.INTER_CUBIC) 
    cap_writer.write(frame)      
    cv2.imshow("frame", frame)
    
    key = cv2.waitKey(1)
    if key == ord('q'):
        cap.release()
        cv2.destroyAllWindows()
        break
    
    success, frame = cap.read()
    frame_num+=1
    


0: 640x640 1 person, 159.9ms
Speed: 3.1ms preprocess, 159.9ms inference, 15.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 person, 151.1ms
Speed: 5.0ms preprocess, 151.1ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)

0: 224x224 no fall 1.00, fall 0.00, 15.6ms
Speed: 0.0ms preprocess, 15.6ms inference, 0.0ms postprocess per image at shape (1, 3, 224, 224)

0: 640x288 1 person, 137.9ms
Speed: 0.0ms preprocess, 137.9ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 288)

0: 640x640 1 person, 147.1ms
Speed: 0.0ms preprocess, 147.1ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

0: 224x224 no fall 1.00, fall 0.00, 17.2ms
Speed: 0.0ms preprocess, 17.2ms inference, 0.0ms postprocess per image at shape (1, 3, 224, 224)

0: 640x288 1 person, 114.6ms
Speed: 2.0ms preprocess, 114.6ms inference, 2.4ms postprocess per image at shape (1, 3, 640, 288)

0: 640x640 1 person, 150.9ms
Speed: 3.2ms preprocess, 150.9ms inference, 0.0m

0.06146072


0: 224x224 no fall 1.00, fall 0.00, 33.7ms
Speed: 0.0ms preprocess, 33.7ms inference, 0.0ms postprocess per image at shape (1, 3, 224, 224)

0: 640x288 1 person, 112.9ms
Speed: 0.0ms preprocess, 112.9ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 288)

0: 640x640 1 person, 163.9ms
Speed: 0.0ms preprocess, 163.9ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)



0.31453788


0: 224x224 no fall 1.00, fall 0.00, 15.7ms
Speed: 0.0ms preprocess, 15.7ms inference, 0.0ms postprocess per image at shape (1, 3, 224, 224)

0: 640x288 1 person, 131.2ms
Speed: 0.0ms preprocess, 131.2ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 288)

0: 640x640 1 person, 111.4ms
Speed: 0.0ms preprocess, 111.4ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)

0: 224x224 no fall 1.00, fall 0.00, 15.5ms
Speed: 0.0ms preprocess, 15.5ms inference, 0.0ms postprocess per image at shape (1, 3, 224, 224)



...

1.7010444


0: 640x640 1 person, 198.1ms
Speed: 0.0ms preprocess, 198.1ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)

0: 224x224 fall 0.99, no fall 0.01, 52.2ms
Speed: 1.3ms preprocess, 52.2ms inference, 0.0ms postprocess per image at shape (1, 3, 224, 224)

0: 640x448 1 person, 225.8ms
Speed: 2.9ms preprocess, 225.8ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 448)

0: 640x640 1 person, 153.2ms
Speed: 13.8ms preprocess, 153.2ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)


0.9352712



0: 224x224 fall 0.99, no fall 0.01, 47.8ms
Speed: 0.0ms preprocess, 47.8ms inference, 0.0ms postprocess per image at shape (1, 3, 224, 224)

0: 640x448 1 person, 200.2ms
Speed: 0.0ms preprocess, 200.2ms inference, 15.4ms postprocess per image at shape (1, 3, 640, 448)


0.20715219


In [28]:
OBJECTS[5]

{'class_id': 0,
 'last_bbox': array([629,  46, 785, 438]),
 'bbox': array([629,  46, 785, 438]),
 'last_kpts': array([[[    0.86116,    0.085251],
         [    0.88868,    0.073328],
         [      0.809,    0.062601],
         [          0,           0],
         [    0.63862,    0.077443],
         [     0.7861,     0.21983],
         [    0.44205,     0.19901],
         [     0.7075,     0.36244],
         [    0.25732,     0.35608],
         [    0.81865,     0.47917],
         [    0.25989,     0.51474],
         [    0.59688,     0.51486],
         [    0.37012,     0.50597],
         [    0.43138,     0.71589],
         [    0.25229,     0.71798],
         [    0.31254,     0.87726],
         [    0.15379,     0.89815]]], dtype=float32),
 'kpts': array([[[    0.86116,    0.085251],
         [    0.88868,    0.073328],
         [      0.809,    0.062601],
         [          0,           0],
         [    0.63862,    0.077443],
         [     0.7861,     0.21983],
         [   

### Frames subtraction of ROI

In [5]:
class Fallen_ROIS:
    def __init__(self):
        self.ROIs = {}
        
        
        
    def add_first(self, object_id, ROI, bbox, label, second_label):
       self.ROIs[object_id]={
            'ROI':ROI,
            'bbox':bbox,
            'label':label,
            'second_label': second_label,
            'not_moving': 0,
            'detected': True
            }    
                    
      
        
    def update(self, object_id, ROI, bbox, label, second_label):
        object = self.ROIs[object_id]
        object['ROI']=ROI
        object['bbox']=bbox
        object['label']=label
        object['second_label']= second_label
        object['detected'] = True 
           
           
            
        
    def compute_diff(self, object_id, ROI):
        '''
        compute difference between previous ROI and ROI from the current frame
        '''
        
        # cv2.imshow("prev ROI", self.ROIs[object_id]['ROI'])
        # cv2.waitKey(0)
        # cv2.destroyAllWindows()
        
        
        # cv2.imshow("current ROI", ROI)
        # cv2.waitKey(0)
        # cv2.destroyAllWindows()
                
        diff = cv2.absdiff(self.ROIs[object_id]['ROI'], ROI) 
        
        #thresh_diff = cv2.threshold(diff, 15, 255, cv2.THRESH_BINARY)[1]
        gray = cv2.cvtColor(diff, cv2.COLOR_BGR2GRAY)
        
        total_pixels = ROI.shape[0] * ROI.shape[1] * 1.0
        
        diff_on_pixels = cv2.countNonZero(gray) * 1.0
        difference_measure = diff_on_pixels / total_pixels
        
        if difference_measure<=0.5:
            self.ROIs[object_id]['not_moving']+=1
        else:
            self.ROIs[object_id]['not_moving']=self.ROIs[object_id]['not_moving']-1 if self.ROIs[object_id]['not_moving'] > 0 else 0
        
        return difference_measure
    
    def remove_old_rois(self):
        rois_copy = self.ROIs.copy()
        for object_id in rois_copy.keys():
            if self.ROIs[object_id]["detected"] == False:
                del self.ROIs[object_id]
                
    def check_not_moving(self, frame):
        for ind, object in self.ROIs.items():
            if object['not_moving']>=MAX_NOT_MOVING:
                cv2.putText(frame, "Not moving!", (150, 50), 
                             cv2.FONT_HERSHEY_SIMPLEX , fontScale = 1, 
                             color=(0, 0, 255), thickness=3)
            print(f"not_moving for object {ind}", object['not_moving'])
            
    def set_detected_false(self):
        for object in self.ROIs.values():
            object["detected"] = False
                             
                

In [8]:
image = cv2.imread("frames_not_moving1/frames_not_moving1_1.jpg")

cv2.imshow("image",image)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [18]:
roi1 = image[200:300,200:300]
cv2.imshow("roi1", roi1)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [24]:
roi2 = image[201:301,200:300]
cv2.imshow("roi1", roi2)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [25]:
diff = cv2.absdiff(roi1, roi2)

cv2.imshow("diff", diff)
cv2.waitKey(0)
cv2.destroyAllWindows()

gray = cv2.cvtColor(diff, cv2.COLOR_BGR2GRAY)

cv2.imshow("diff", gray)
cv2.waitKey(0)
cv2.destroyAllWindows()

total_pixels = roi1.shape[0] * roi1.shape[1] * 1.0
        
diff_on_pixels = cv2.countNonZero(gray) * 1.0
print(diff_on_pixels / total_pixels)


0.7056


In [6]:
fallen_rois = Fallen_ROIS()

In [7]:
cap = cv2.VideoCapture("data/people_falling_3.mp4")

frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

scale_percent = 50 # percent of original size
width = int(frame_width * scale_percent / 100)
height = int(frame_height * scale_percent / 100)


cap_writer = cv2.VideoWriter('data/result_detection3.avi', cv2.VideoWriter_fourcc(*'MJPG'),
                        30, (width, height))

success, frame = cap.read()
frame_num=0
total_fps=0


while success:
    
    
    width = int(frame.shape[1] * scale_percent / 100)
    height = int(frame.shape[0] * scale_percent / 100)
    
    dim = (width, height)
  
    # resize image
    frame = cv2.resize(frame, dim, interpolation = cv2.INTER_AREA)
    
        
    alarm = False
    not_moving = False
    
    fallen_rois.set_detected_false() 
       
    if frame_num%1==0:
        #renew predictions every 5 frame
        detections = detector(frame)[0]
        detections = sv.Detections.from_ultralytics(detections)  
                
    detections = byte_tracker.update_with_detections(detections)
        
       
    bboxes = detections.xyxy
    confidences = detections.confidence
    class_ids = detections.class_id
    tracker_ids = detections.tracker_id
    
            
    for pred_box, label, tracker_id in zip(bboxes, class_ids, tracker_ids):
        
        if label == 0:
            bbox = list(map(int, pred_box))
            
            ROI = frame[bbox[1]:bbox[3], 
                    bbox[0]:bbox[2]]
        
            results = classifier(ROI)
            second_label = classes[np.argmax(results[0].probs.data)]
            
            if second_label=="fall":               
                
                if tracker_id not in fallen_rois.ROIs:
                    fallen_rois.add_first(tracker_id, ROI, bbox, label, second_label)
                   
                else:
                    #take previous frame ROI coordinates
                    x1, y1, x2, y2 = fallen_rois.ROIs[tracker_id]['bbox']
                    #extract bbox of the current frame with the coordinates of the previous frame
                    extracted_ROI = frame[y1:y2, x1:x2]
                    
                    diff = fallen_rois.compute_diff(tracker_id, extracted_ROI)
                    
                    print(f"diff for {tracker_id}", diff)
                    fallen_rois.update(tracker_id, ROI, bbox, label, second_label)
                    
            # else:
            #     fallen_rois = Fallen_ROIS()                   
        
            
            cv2.rectangle(frame, (bbox[0]-2, bbox[1]-2), (bbox[2]+2, bbox[3]+2), color=(0,0,255), thickness=2)
            cv2.putText(frame, "id: "+str(tracker_id), (bbox[0], bbox[1]-10), 
                        cv2.FONT_HERSHEY_SIMPLEX , fontScale = 0.5,
                        color=(0, 0, 0), thickness=1)
            cv2.putText(frame, second_label, (bbox[0]+50, bbox[1]-10),
                        cv2.FONT_HERSHEY_SIMPLEX , fontScale = 0.5,
                        color=(0, 0, 0), thickness=1)
            
        
    fallen_rois.remove_old_rois()
        
        #check falling objects
    fallen_rois.check_not_moving(frame)    
        
             
                
                    
    #resize frame for videowriter
    #frame = cv2.resize(frame, dsize=(640, 640), interpolation=cv2.INTER_CUBIC) 
    cap_writer.write(frame)      
    cv2.imshow("frame", frame)
    
    key = cv2.waitKey(1)
    if key == ord('q'):
        cap.release()
        cv2.destroyAllWindows()
        break
    
    success, frame = cap.read()
    frame_num+=1
    


Loading models\yolov8s_people.onnx for ONNX Runtime inference...

0: 640x640 3 persons, 147.4ms
Speed: 0.0ms preprocess, 147.4ms inference, 15.6ms postprocess per image at shape (1, 3, 640, 640)

0: 224x224 no fall 0.83, fall 0.17, 40.1ms
Speed: 0.0ms preprocess, 40.1ms inference, 0.0ms postprocess per image at shape (1, 3, 224, 224)

0: 224x224 fall 1.00, no fall 0.00, 17.7ms
Speed: 0.0ms preprocess, 17.7ms inference, 0.0ms postprocess per image at shape (1, 3, 224, 224)

0: 224x224 no fall 1.00, fall 0.00, 22.7ms
Speed: 0.0ms preprocess, 22.7ms inference, 0.0ms postprocess per image at shape (1, 3, 224, 224)



not_moving for object 2 0


0: 640x640 3 persons, 158.2ms
Speed: 3.3ms preprocess, 158.2ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)

0: 224x224 no fall 0.89, fall 0.11, 41.2ms
Speed: 0.0ms preprocess, 41.2ms inference, 0.0ms postprocess per image at shape (1, 3, 224, 224)

0: 224x224 fall 1.00, no fall 0.00, 21.8ms
Speed: 2.0ms preprocess, 21.8ms inference, 1.0ms postprocess per image at shape (1, 3, 224, 224)

0: 224x224 no fall 1.00, fall 0.00, 21.1ms
Speed: 0.0ms preprocess, 21.1ms inference, 0.0ms postprocess per image at shape (1, 3, 224, 224)



diff for 2 0.3278037904124861
not_moving for object 2 1


0: 640x640 3 persons, 156.1ms
Speed: 9.5ms preprocess, 156.1ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)

0: 224x224 no fall 0.87, fall 0.13, 21.4ms
Speed: 2.0ms preprocess, 21.4ms inference, 0.0ms postprocess per image at shape (1, 3, 224, 224)

0: 224x224 fall 1.00, no fall 0.00, 20.6ms
Speed: 1.0ms preprocess, 20.6ms inference, 0.0ms postprocess per image at shape (1, 3, 224, 224)

0: 224x224 no fall 1.00, fall 0.00, 10.7ms
Speed: 1.0ms preprocess, 10.7ms inference, 0.0ms postprocess per image at shape (1, 3, 224, 224)

0: 640x640 3 persons, 157.5ms


...

diff for 1 0.6095773396658353
diff for 14 0.7060888451145289
diff for 13 0.7369912877431675
not_moving for object 1 11
not_moving for object 14 0
not_moving for object 13 2


0: 640x640 3 persons, 204.0ms
Speed: 4.0ms preprocess, 204.0ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

0: 224x224 fall 1.00, no fall 0.00, 64.6ms
Speed: 1.0ms preprocess, 64.6ms inference, 0.0ms postprocess per image at shape (1, 3, 224, 224)

0: 224x224 fall 1.00, no fall 0.00, 27.5ms
Speed: 5.4ms preprocess, 27.5ms inference, 0.0ms postprocess per image at shape (1, 3, 224, 224)

0: 224x224 fall 1.00, no fall 0.00, 29.3ms
Speed: 1.3ms preprocess, 29.3ms inference, 0.0ms postprocess per image at shape (1, 3, 224, 224)



diff for 1 0.6105729166666667
diff for 14 0.7157614213197969
diff for 13 0.7561946102369326
not_moving for object 1 10
not_moving for object 14 0
not_moving for object 13 1


0: 640x640 3 persons, 260.9ms
Speed: 5.0ms preprocess, 260.9ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

0: 224x224 fall 1.00, no fall 0.00, 47.8ms
Speed: 1.0ms preprocess, 47.8ms inference, 0.0ms postprocess per image at shape (1, 3, 224, 224)

0: 224x224 fall 1.00, no fall 0.00, 34.9ms
Speed: 1.0ms preprocess, 34.9ms inference, 0.0ms postprocess per image at shape (1, 3, 224, 224)

0: 224x224 fall 1.00, no fall 0.00, 22.9ms
Speed: 1.0ms preprocess, 22.9ms inference, 0.0ms postprocess per image at shape (1, 3, 224, 224)


diff for 1 0.6257704624496402
diff for 14 0.718003281714696
diff for 13 0.7636227320437847
not_moving for object 1 9
not_moving for object 14 0
not_moving for object 13 0


In [8]:
for ind, object in fallen_rois.ROIs.items():
    print(ind)
    print(object['not_moving'])

1
9
14
0
13
0


In [20]:
fallen_rois.ROIs

{6: {'ROI': array([[[ 91, 107, 126],
          [ 93, 109, 128],
          [ 91, 107, 126],
          ...,
          [ 87, 102, 118],
          [ 86, 101, 117],
          [ 84,  99, 115]],
  
         [[ 93, 112, 130],
          [ 92, 111, 129],
          [ 93, 112, 130],
          ...,
          [ 87, 102, 118],
          [ 88, 103, 119],
          [ 86, 101, 117]],
  
         [[ 93, 112, 130],
          [ 94, 113, 131],
          [ 94, 113, 131],
          ...,
          [ 86, 101, 117],
          [ 88, 103, 119],
          [ 87, 102, 118]],
  
         ...,
  
         [[198, 213, 229],
          [201, 215, 229],
          [200, 214, 228],
          ...,
          [113, 136, 157],
          [113, 136, 157],
          [113, 136, 155]],
  
         [[198, 212, 231],
          [200, 214, 228],
          [200, 214, 228],
          ...,
          [115, 138, 159],
          [113, 136, 155],
          [111, 136, 155]],
  
         [[197, 211, 230],
          [199, 213, 227],
          [199